# Creación de capas personalizadas

En ciertas ocasiones es posible que se desee implementar una nueva capa de la red neuronal artificial para la que no existe una implementación en Tensorflow o Keras. En estos casos podemos utilizar Tensorflow para crear capas personalizadas.

### Importando el conjunto de datos

In [1]:
from tensorflow.keras import datasets

In [2]:
boston_housing = datasets.boston_housing

In [3]:
(X_train, y_train), (X_test, y_test) = boston_housing.load_data()

In [4]:
import pandas as pd
import numpy as np

features = ["CRIM", "ZN", "INDUS", "CHAS", "NOX", "RM", "AGE", "DIS", "RAD", "TAX", "PTRATIO", "B", "LSTAT", "MEDV"]

df_train = pd.DataFrame(np.column_stack([X_train, y_train]), columns=features)
df_train.head(10)

,CRIM,ZN,INDUS,CHAS,NOX,RM,AGE,DIS,RAD,TAX,PTRATIO,B,LSTAT,MEDV
0,1.23247,0.0,8.14,0.0,0.538,6.142,91.7,3.9769,4.0,307.0,21.0,396.90,18.72,15.2
1,0.02177,82.5,2.03,0.0,0.415,7.610,15.7,6.2700,2.0,348.0,14.7,395.38,3.11,42.3
2,4.89822,0.0,18.10,0.0,0.631,4.970,100.0,1.3325,24.0,666.0,20.2,375.52,3.26,50.0
3,0.03961,0.0,5.19,0.0,0.515,6.037,34.5,5.9853,5.0,224.0,20.2,396.90,8.01,21.1
4,3.69311,0.0,18.10,0.0,0.713,6.376,88.4,2.5671,24.0,666.0,20.2,391.43,14.65,17.7
5,0.28392,0.0,7.38,0.0,0.493,5.708,74.3,4.7211,5.0,287.0,19.6,391.13,11.74,18.5
6,9.18702,0.0,18.10,0.0,0.700,5.536,100.0,1.5804,24.0,666.0,20.2,396.90,23.60,11.3
7,4.09740,0.0,19.58,0.0,0.871,5.468,100.0,1.4118,5.0,403.0,14.7,396.90,26.42,15.6
8,2.15505,0.0,19.58,0.0,0.871,5.628,100.0,1.5166,5.0,403.0,14.7,169.27,16.65,15.6
9,1.62864,0.0,21.89,0.0,0.624,5.019,100.0,1.4394,4.0,437.0,21.2,396.90,34.41,14.4


### División del conjunto de datos

In [5]:
from sklearn.model_selection import train_test_split

X_test, X_val, y_test, y_val = train_test_split(X_test, y_test, test_size=0.5)

### Escalando el conjunto de datos

In [6]:
from sklearn.preprocessing import RobustScaler

scaler = RobustScaler()

X_train_prep = scaler.fit_transform(X_train)
X_val_prep = scaler.transform(X_val)
X_test_prep = scaler.transform(X_test)

### Definicion de la arquitectura de la Red Neuronal Artificial

### Creación de capas sin parámetros

Las capas sin parámetros aplican transformaciones sobre los valores de entrada. Algunos ejemplos de capas de este tipo que se encuentran implementados en Keras son: _keras.layers.Flatten_ o _keras.layers.ReLU_. 

En estos casos, la mejor forma de crear este tipo de capas es desarrollar una función personalizada y utilizarla junto con la construcción _keras.layers.Lambda_.

In [7]:
from tensorflow import keras
import tensorflow as tf

# Capa personalizada que eleva al cuadrado los valores de entrada
square_layer = keras.layers.Lambda(lambda x: tf.square(x))

Esta capa se utiliza de la misma forma que el resto de capas que se encuentran implementadas en Keras por defecto

In [8]:
from tensorflow.keras import models
from tensorflow.keras import layers

In [9]:
network = models.Sequential()

network.add(layers.Dense(30, activation='relu', input_shape=X_train.shape[1:]))
network.add(layers.Dense(10, activation='relu'))
network.add(square_layer)
network.add(layers.Dense(1))

In [10]:
network.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense (Dense)               (None, 30)                420       
                                                                 
 dense_1 (Dense)             (None, 10)                310       
                                                                 
 lambda (Lambda)             (None, 10)                0         
                                                                 
 dense_2 (Dense)             (None, 1)                 11        
                                                                 
Total params: 741 (2.89 KB)
Trainable params: 741 (2.89 KB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


In [11]:
network.compile(
    loss='mean_squared_error',
    optimizer='adam',
    metrics=['mae'])

In [12]:
history = network.fit(X_train_prep, 
                      y_train, 
                      epochs=50, 
                      validation_data=(X_val_prep, y_val))

Epoch 1/50
13/13 [==============================] - 1s 27ms/step - loss: 623.7374 - mae: 23.2361 - val_loss: 587.2205 - val_mae: 22.4653
Epoch 2/50
13/13 [==============================] - 0s 4ms/step - loss: 596.5098 - mae: 22.6389 - val_loss: 570.7358 - val_mae: 22.0856
Epoch 3/50
13/13 [==============================] - 0s 5ms/step - loss: 578.6555 - mae: 22.2073 - val_loss: 554.4739 - val_mae: 21.7175
Epoch 4/50
13/13 [==============================] - 0s 5ms/step - loss: 557.4581 - mae: 21.7056 - val_loss: 530.5547 - val_mae: 21.1575
Epoch 5/50
13/13 [==============================] - 0s 4ms/step - loss: 526.0392 - mae: 20.8643 - val_loss: 492.3597 - val_mae: 20.1862
Epoch 6/50
13/13 [==============================] - 0s 4ms/step - loss: 481.8240 - mae: 19.9137 - val_loss: 447.3049 - val_mae: 19.1664
Epoch 7/50
13/13 [==============================] - 0s 5ms/step - loss: 429.4630 - mae: 18.8110 - val_loss: 398.9543 - val_mae: 17.8593
Epoch 8/50
13/13 [=============================

In [13]:
network.predict(X_test_prep)

2/2 [==============================] - 0s 5ms/step


array([[22.760721],
       [20.390284],
       [23.223104],
       [19.164227],
       [37.117233],
       [20.965393],
       [20.302502],
       [16.84701 ],
       [23.275455],
       [25.78372 ],
       [26.059137],
       [21.670778],
       [59.874493],
       [24.749012],
       [15.68661 ],
       [27.712446],
       [21.64017 ],
       [25.134048],
       [20.752739],
       [11.424868],
       [ 8.082136],
       [15.226792],
       [17.312689],
       [49.354935],
       [27.907757],
       [32.77675 ],
       [23.165483],
       [11.31893 ],
       [41.788773],
       [ 7.276226],
       [22.319553],
       [16.659508],
       [32.284847],
       [18.30059 ],
       [14.889567],
       [21.172192],
       [38.529568],
       [14.002282],
       [23.03311 ],
       [14.44152 ],
       [29.86906 ],
       [20.672985],
       [21.270094],
       [23.04155 ],
       [34.428493],
       [33.239475],
       [18.843918],
       [34.66697 ],
       [21.911268],
       [39.510468],
